In [1]:
import os
import time
import pandas as pd
import json
from groq import Groq
from dotenv import load_dotenv

# --- CONFIGURATION ---
# 1. Chargement des variables d'environnement (.env)
load_dotenv()
API_KEY = os.getenv("GROQ_API_KEY")

if not API_KEY:
    raise ValueError("Erreur : La clé API 'GROQ_API_KEY' est introuvable dans le fichier .env")

# 2. Paramètres du script
OUTPUT_FILE = "dataset_marrakech_english.csv"
OBJECTIF_TOTAL = 10000
BATCH_SIZE = 20  # Générer 20 avis par appel (compromis vitesse/stabilité)
MODEL_NAME = "llama-3.3-70b-versatile" # Modèle très performant sur Groq

# Initialisation du client
client = Groq(api_key=API_KEY)

def get_last_id():
    """
    Vérifie si le fichier CSV existe déjà et retourne le dernier ID.
    Cela permet de relancer le script sans tout recommencer.
    """
    if os.path.exists(OUTPUT_FILE):
        try:
            df = pd.read_csv(OUTPUT_FILE)
            if not df.empty and 'ID' in df.columns:
                return int(df['ID'].max())
        except Exception as e:
            print(f"Attention: Fichier existant illisible ({e}). On recommence à 0.")
    return 0

def generate_english_reviews(batch_size, start_id):
    """
    Appelle l'API Groq pour générer un lot d'avis en anglais.
    """
    
    # Prompt optimisé pour Llama 3 en anglais
    prompt = f"""
    You are a data generator for sentiment analysis.
    Generate {batch_size} realistic tourist reviews about Marrakech, Morocco.
    
    Strict Constraints:
    1. Language: English (casual, natural, sometimes using slang like 'wow', 'meh').
    2. Topics: Jemaa el-Fna square, Majorelle Garden, Hotels/Riads, Heat, Scams/Taxis, Food/Tagine, Architecture, Atlas Mountains, Locals Hospitality.
    3. Sentiment Distribution: Balanced mix of Positive, Negative, and Neutral.
    4. Style: Short to medium length (1 to 3 sentences). Use emojis moderately (1-3 max).
    5. Output Format: Return ONLY a valid JSON object with a single key "reviews" containing the list.
    
    Expected JSON Structure:
    {{
        "reviews": [
            {{"review": "The garden was beautiful but way too crowded. 🌵", "sentiment_label": "Neutral"}},
            {{"review": "Best tagine I've ever had! The staff was lovely. 😍", "sentiment_label": "Positive"}},
            {{"review": "Scammed by a taxi driver, ruined my mood. 😡", "sentiment_label": "Negative"}}
        ]
    }}
    """

    try:
        completion = client.chat.completions.create(
            messages=[
                {"role": "system", "content": "You are a helpful assistant that outputs strict JSON."},
                {"role": "user", "content": prompt}
            ],
            model=MODEL_NAME,
            temperature=0.85, # Créativité élevée pour éviter les répétitions
            response_format={"type": "json_object"}, # Force le JSON valide
        )
        
        # Parsing de la réponse
        json_content = completion.choices[0].message.content
        data = json.loads(json_content)
        
        # Gestion flexible de la structure JSON (parfois Llama oublie la clé racine)
        reviews_list = data.get('reviews', [])
        
        # Ajout des IDs et formatage
        formatted_data = []
        current_id = start_id
        
        for item in reviews_list:
            current_id += 1
            formatted_data.append({
                "ID": current_id,
                "review": item.get('review'),
                "sentiment_label": item.get('sentiment_label')
            })
            
        return formatted_data

    except Exception as e:
        print(f"  [Erreur API] : {e}")
        return []

def main():
    print(f"--- Synthetic Data Generator (Marrakech / English) ---")
    print(f"Model: {MODEL_NAME}")
    print(f"Target: {OBJECTIF_TOTAL} reviews")
    
    # 1. Gestion de la reprise (Resume capability)
    last_id = get_last_id()
    current_count = last_id
    
    if last_id > 0:
        print(f"Reprise détectée à l'ID : {last_id}")
    else:
        # Création du fichier avec en-têtes si nouveau
        pd.DataFrame(columns=["ID", "review", "sentiment_label"]).to_csv(OUTPUT_FILE, index=False)
        print("Création d'un nouveau fichier de données.")

    # 2. Boucle principale
    while current_count < OBJECTIF_TOTAL:
        remaining = OBJECTIF_TOTAL - current_count
        current_batch_size = min(BATCH_SIZE, remaining)
        
        print(f"Génération batch (IDs {current_count + 1} à {current_count + current_batch_size})...")
        
        new_data = generate_english_reviews(current_batch_size, current_count)
        
        if new_data:
            # Sauvegarde immédiate (Append mode)
            df_batch = pd.DataFrame(new_data)
            df_batch.to_csv(OUTPUT_FILE, mode='a', header=False, index=False)
            
            current_count += len(new_data)
            print(f"  > Sauvegardé. Total: {current_count}/{OBJECTIF_TOTAL}")
            
            # Pause anti-ban (Rate limit Groq)
            time.sleep(1.5) 
        else:
            print("  ! Batch vide ou erreur. Pause de 5s avant nouvelle tentative...")
            time.sleep(5)

    print(f"\nTerminé ! {OBJECTIF_TOTAL} avis sauvegardés dans '{OUTPUT_FILE}'.")

if __name__ == "__main__":
    main()

--- Synthetic Data Generator (Marrakech / English) ---
Model: llama-3.3-70b-versatile
Target: 10000 reviews
Reprise détectée à l'ID : 4506
Génération batch (IDs 4507 à 4526)...
  > Sauvegardé. Total: 4526/10000
Génération batch (IDs 4527 à 4546)...
  > Sauvegardé. Total: 4546/10000
Génération batch (IDs 4547 à 4566)...
  > Sauvegardé. Total: 4567/10000
Génération batch (IDs 4568 à 4587)...
  > Sauvegardé. Total: 4587/10000
Génération batch (IDs 4588 à 4607)...
  > Sauvegardé. Total: 4607/10000
Génération batch (IDs 4608 à 4627)...
  > Sauvegardé. Total: 4628/10000
Génération batch (IDs 4629 à 4648)...
  > Sauvegardé. Total: 4648/10000
Génération batch (IDs 4649 à 4668)...
  > Sauvegardé. Total: 4668/10000
Génération batch (IDs 4669 à 4688)...
  > Sauvegardé. Total: 4688/10000
Génération batch (IDs 4689 à 4708)...
  > Sauvegardé. Total: 4708/10000
Génération batch (IDs 4709 à 4728)...
  > Sauvegardé. Total: 4728/10000
Génération batch (IDs 4729 à 4748)...
  > Sauvegardé. Total: 4748/100

KeyboardInterrupt: 